# Euclidean Time Evolution QMC
We can use a strategy similar to GFMC to randomly thermalize a given timeslice using weights and resampling to acquire a representative Monte Carlo sample for the ground state. This doesn't provide a nice way to access finite temperature, but could be useful for studying quantum critical points or other zero-temperature properties.

In [ ]:
import analysis as al
import matplotlib.pyplot as plt
import numpy as np
import tqdm.auto as tqdm
%matplotlib widget

# Square lattice

### Hamiltonian
The only operator appearing in the Hamiltonian takes the form
$$
-J P_{wxyz} X_w X_x X_y X_z
$$
which is a controlled $X$ operator dependent on the orientation of the four links.
In the height representation, we have
$$
-J P_{abcd} X_{p}
$$
where $a$, $b$, $c$, $d$ are the neighboring plaquettes and $p$ is the current plaquette.

### Time evolution
Exponentiating this operator times $dt$ gives the Trotterized time evolution operator.
$$
\exp(dt J P X) = (1-P) + P [\cosh(dt J) + \sinh(dt J) X] = (1-P) + P N [(1-p) + p X],
$$
where $p = e^{-dt J} \sinh(dt J)$, $1-p = e^{-dt J} \cosh(dt J)$, and $N = e^{dt J}$.


In [ ]:
def make_eo_mask(shape):
    i, j = np.ix_(np.arange(shape[0]), np.arange(shape[1]))
    return (i + j) % 2 == 0

In [ ]:
def measure_Fx(x):
    a = np.roll(x, -1, axis=0)
    b = np.roll(x, 1, axis=0)
    c = np.roll(x, -1, axis=1)
    d = np.roll(x, 1, axis=1)
    return (a == b) & (b == c) & (c == d)

In [ ]:
def step(x, *, dtJ1, dtJ2):
    logw = 0.0
    def update_sublattice(dtJ):
        nonlocal logw
        a = np.roll(x, -1, axis=0)
        b = np.roll(x, 1, axis=0)
        c = np.roll(x, -1, axis=1)
        d = np.roll(x, 1, axis=1)
        P = (a == b) & (b == c) & (c == d)
        logw += np.sum(np.where(mask & P, -dtJ, 0.0))
        p = np.where(P, np.sinh(dtJ)*np.exp(-dtJ), 0.0)
        r = np.random.random(size=x.shape)
        x[mask & (r < p)] *= -1
    # even sites
    mask = make_eo_mask(x.shape)
    update_sublattice(dtJ1)
    # odd sites
    mask = ~mask
    update_sublattice(dtJ2)
    return logw

In [ ]:
def resample(logw):
    assert len(logw.shape) == 1
    # average weight
    new_logw = np.logaddexp.reduce(logw) - np.log(len(logw))
    logw -= new_logw + np.log(len(logw))
    assert np.isclose(np.logaddexp.reduce(logw), 0.0), np.logaddexp.reduce(logw)
    inds = np.random.choice(np.arange(len(logw)), size=len(logw), p=np.exp(logw))
    return inds, new_logw

In [ ]:
def measure_Mx(x):
    mask = make_eo_mask(x.shape)
    return 2*np.mean(x * mask), 2*np.mean(x * (1-mask))

In [ ]:
def run_walkers(x, *, dtJ1, dtJ2, n_iter, n_resample, n_meas):
    logw = np.zeros(x.shape[0])
    hist = dict(logw=[], Mlogw=[], Mx=[], cfgs=[])
    for i in tqdm.tqdm(range(n_iter)):
        for j,xj in enumerate(x):
            logw[j] += step(xj, dtJ1=dtJ1, dtJ2=dtJ2)
        if (i+1) % n_resample == 0:
            inds, new_logw = resample(logw)
            # print(f'Variety: {len(np.unique(inds))/len(inds)}')
            x = x[inds]
            logw[:] = new_logw
        if (i+1) % n_meas == 0:
            # np.stack([measure_Fx(xj) for xj in x])
            hist['cfgs'].append(np.copy(x))
            hist['Mx'].append(np.stack([measure_Mx(xj) for xj in x]))
            hist['Mlogw'].append(np.copy(logw))
        hist['logw'].append(np.copy(logw))
    return hist

In [ ]:
x = -np.ones((128, 64, 64))
hist = run_walkers(x, dtJ1=0.01, dtJ2=0.05, n_iter=10000, n_resample=10, n_meas=10)

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(6,3))
# logw = al.bootstrap(np.stack(hist['logw'], axis=1), Nboot=1000, f=al.rmean)
# al.add_errorbar(logw, ax=ax)
for logwi in np.stack(hist['logw'], axis=1):
    axes[0].plot(logwi, linewidth=0.5, color='0.5')
def weighted_meas(logw, M):
    logw = logw - np.max(logw, axis=0)
    return al.rmean(np.exp(logw)*np.abs(M)) / al.rmean(np.exp(logw))
Mx = np.stack(al.bootstrap(
    np.stack(hist['Mlogw'], axis=1)[...,None], np.stack(hist['Mx'], axis=1),
    Nboot=100, f=weighted_meas))
print(f'{Mx.shape=}')
al.add_errorbar(Mx[:,:,0], xs=np.arange(0, 10000, 10), ax=axes[1], label='Mx even')
al.add_errorbar(Mx[:,:,1], xs=np.arange(0, 10000, 10), ax=axes[1], label='Mx odd')
axes[1].legend()
plt.show()

In [ ]:
fig, axes = plt.subplots(2,4)
mask = make_eo_mask(hist['cfgs'][0][0].shape)
cmap = plt.get_cmap('Grays')
cmap.set_bad(alpha=0.0)
for ax_col,cfg in zip(np.transpose(axes),np.stack(hist['cfgs'])[-4:,0]):
    cfg_even = np.where(mask, cfg, float('nan'))
    cfg_odd = np.where(~mask, cfg, float('nan'))
    ax_col[0].imshow(cfg_even, interpolation='nearest', cmap=cmap)
    ax_col[1].imshow(cfg_odd, interpolation='nearest', cmap=cmap)
    ax.set_aspect(1.0)
plt.show()